# template から HTML と JS を生成する


In [ ]:
# Move up to top directory
import os
from pathlib import Path

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/others/IMPC-project/notebooks/notebools-web
/mnt/e/Research/others/IMPC-project


# 古いファイルの消去


In [8]:
%%bash

rm -rf web/network/phenotype
rm -rf web/network/genesymbol
rm -rf web/network/data

mkdir -p web/network/phenotype
mkdir -p web/network/genesymbol
mkdir -p web/network/data

echo "*" > web/network/phenotype/.gitignore
echo "*" > web/network/genesymbol/.gitignore
echo "*" > web/network/data/.gitignore


## index.htmlの描画に必要なファイルを生成する


In [5]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    # print(path_mp_term.stem)
    if not Path("data", "network", f"{mp_term}.json").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    # print(mp_term_name_space)
    mp_terms[mp_term_name_space] = mp_term

In [6]:
print(list(mp_terms)[:3])
print(len(mp_terms))

[]
0


In [ ]:
import json

json.dump(mp_terms, open("data/available_mp_terms.json", "w"), indent=2)

In [13]:
%%bash

cp data/available_mp_terms.json web/network/data/available_mp_terms.json
cp data/available_gene_symbols.txt web/network/data/available_gene_symbols.txt
cp data/marker_symbol_accession_id.json web/network/data/marker_symbol_accession_id.json


## index.htmlを生成

In [39]:
%%bash
grep -v "REMOVE_THIS_LINE" web/network/template/template_index.html > web/index.html

# JSON のコピー


In [16]:
%%bash
mkdir -p web/network/phenotype/data
mkdir -p web/network/genesymbol/data

cp data/network/phenotype/* web/network/phenotype/data
cp data/network/genesymbol/* web/network/genesymbol/data

# Template から HTML と JS を生成


In [37]:
%%bash

########################################
# Phenotype
########################################
mkdir -p web/network/phenotype/js

find data/mp_term_name -type f |
sed "s|data/mp_term_name/||" |
sed "s|.csv$||" |
while read mp_term_name_underscore; do
    mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
    impc_url=$(grep "$mp_term_name_space" data/mp_term_name_id_url.tsv | cut -f 2)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat web/network/template/template_network_phenotype.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/phenotype/"$mp_term_name_underscore".html

    # Javascript
    cat web/network/template/template_network_phenotype.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > web/network/phenotype/js/"$mp_term_name_underscore".js
done
# 11.4s

sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command
sed: -e expression #1, char 77: unterminated `s' command


In [36]:
%%bash

########################################
# Gene Symbol
########################################
mkdir -p web/network/genesymbol/js

cat data/available_gene_symbols.txt |
while read gene_symbol; do
    impc_url="https://www.mousephenotype.org/data/genes/"$(grep "$gene_symbol" data/marker_symbol_accession_id.tsv | cut -f 2)
    escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
    # HTML
    cat web/network/template/template_network_genesymbol.html |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_impc_url|${escaped_url}|g" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > web/network/genesymbol/"$gene_symbol".html

    # Javascript
    cat web/network/template/template_network_genesymbol.js |
    grep -v "REMOVE_THIS_LINE" |
    sed "s|XXX_genesymbol|${gene_symbol}|g" > web/network/genesymbol/js/"$gene_symbol".js
done

# 1 min 10 sec

sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 71: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 71: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 72: unterminated `s' command
sed: -e expression #1, char 73: unterminated `s' command
sed: -e expression #1, char 73: